In [ ]:
# NOTEBOOK 1
!pip install gdown --quiet
!pip install mlflow scikit-learn pyngrok --quiet

import os
import gdown
import zipfile
import random
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import mlflow
from pyngrok import ngrok
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Configurar semillas para reproducibilidad
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Descargar y preparar dataset
file_id_train = '1N1cLDx1jACfSt7lqO5gnQ--7f6rXOiLs'
file_id_test = '1mHE56atoeuUlkZjBR24dVtmhrN18dpoO'
output_train = 'train.zip'
output_test = 'test.zip'

if not os.path.exists('train'):
    print("Descargando train.zip...")
    gdown.download(id=file_id_train, output=output_train, quiet=False)
    with zipfile.ZipFile(output_train, 'r') as zip_ref:
        zip_ref.extractall('train')

if not os.path.exists('test'):
    print("Descargando test.zip...")
    gdown.download(id=file_id_test, output=output_test, quiet=False)
    with zipfile.ZipFile(output_test, 'r') as zip_ref:
        zip_ref.extractall('test')

# Cargar datasets
ds_train_ = tf.keras.preprocessing.image_dataset_from_directory(
    'train',
    labels='inferred',
    label_mode='binary',
    image_size=(64, 64),
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

ds_valid_ = tf.keras.preprocessing.image_dataset_from_directory(
    'test',
    labels='inferred',
    label_mode='binary',
    image_size=(64, 64),
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

# Preprocesamiento de datos
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.AUTOTUNE

ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

# Guardar datasets preprocesados para usar en los siguientes notebooks
tf.data.experimental.save(ds_train, 'preprocessed_train')
tf.data.experimental.save(ds_valid, 'preprocessed_valid')